In [ ]:
pip install transformers datasets evaluate torch

In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from datasets import load_dataset
import evaluate
import torch

In [2]:
dataset = load_dataset("squad")
val_data = dataset["validation"].select(range(50))  # For quicker comparison

In [3]:
def get_answer(model, tokenizer, question, context):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    start_idx = torch.argmax(outputs.start_logits)
    end_idx = torch.argmax(outputs.end_logits) + 1
    input_ids = inputs["input_ids"][0][start_idx:end_idx]
    return tokenizer.decode(input_ids, skip_special_tokens=True)


In [5]:
model_paths = [
    "bert-squad",
    "distilbert-squad",
    "albert-squad",
]

models = []
tokenizers = []

for path in model_paths:
    tokenizer = AutoTokenizer.from_pretrained(path)
    model = AutoModelForQuestionAnswering.from_pretrained(path)
    models.append(model)
    tokenizers.append(tokenizer)


In [6]:
metric = evaluate.load("squad")
results = []

for idx, (model, tokenizer) in enumerate(zip(models, tokenizers)):
    predictions = []
    references = []
    
    for sample in val_data:
        pred = get_answer(model, tokenizer, sample["question"], sample["context"])
        predictions.append({"id": sample["id"], "prediction_text": pred})
        references.append({"id": sample["id"], "answers": sample["answers"]})
    
    score = metric.compute(predictions=predictions, references=references)
    results.append((f"Model {idx+1}", score))


In [7]:
for name, score in results:
    print(f"{name} - Exact Match: {score['exact_match']:.2f}, F1: {score['f1']:.2f}")


Model 1 - Exact Match: 68.00, F1: 73.74
Model 2 - Exact Match: 64.00, F1: 71.73
Model 3 - Exact Match: 84.00, F1: 85.60


In [ ]:
import torch
import time
import numpy as np
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
from sentence_transformers import SentenceTransformer, util

# Load models
qa_models = {
    "BERT": "bert-squad",
    "DistilBERT": "distilbert-squad",
    "ALBERT": "albert-squad",
}

qa_pipelines = {name: pipeline("question-answering", model=model_name, tokenizer=model_name)
                for name, model_name in qa_models.items()}

# Load sentence similarity model
similarity_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Shared context for all questions
shared_context = """Our platform allows users to manage their accounts easily. 
To reset a password, go to 'Account Settings' and select 'Forgot password'. 
To change your email, visit 'Account Settings' > 'Edit Profile'. 
Account suspension occurs due to policy violations. For details, check your email or contact support. 
To delete your account, go to 'Settings' > 'Account' and click 'Delete Account'. This action is irreversible. 
If you need help, visit our Help Center or contact customer support. 
Subscription plans can be upgraded or downgraded in 'Billing' under 'Settings'. 
Refund requests must be submitted within 14 days of purchase. Processing takes 5-7 business days. 
Two-factor authentication (2FA) can be enabled in 'Security Settings' for added account protection. 
Your data is encrypted and follows industry-standard security protocols."""

# Bulk dataset (Expanded Questions + Correct Answers)
qa_tests = [
    {"question": "How do I reset my password?", "correct_answer": "Go to 'Account Settings' and select 'Forgot password'."},
    {"question": "How can I change my email address?", "correct_answer": "Visit 'Account Settings' > 'Edit Profile'."},
    {"question": "Why is my account suspended?", "correct_answer": "Suspension occurs due to policy violations. Check your email or contact support."},
    {"question": "How do I delete my account?", "correct_answer": "Go to 'Settings' > 'Account' and click 'Delete Account'."},
    {"question": "Where can I find help if I have issues?", "correct_answer": "Visit our Help Center or contact customer support."},
    {"question": "How do I upgrade my subscription?", "correct_answer": "Upgrade in 'Billing' under 'Settings'."},
    {"question": "Can I request a refund?", "correct_answer": "Refunds must be requested within 14 days of purchase. Processing takes 5-7 business days."},
    {"question": "How do I enable two-factor authentication?", "correct_answer": "Enable 2FA in 'Security Settings'."},
    {"question": "How is my data protected?", "correct_answer": "Your data is encrypted and follows industry-standard security protocols."},
    {"question": "How do I downgrade my subscription?", "correct_answer": "Downgrade in 'Billing' under 'Settings'."},
    {"question": "How long does it take to process a refund?", "correct_answer": "Refund processing takes 5-7 business days."},
    {"question": "Where can I check my billing details?", "correct_answer": "Billing details can be found in 'Billing' under 'Settings'."},
    {"question": "What should I do if I forgot my password?", "correct_answer": "Go to 'Account Settings' and select 'Forgot password'."},
    {"question": "What happens if I delete my account?", "correct_answer": "Deleting your account is irreversible."},
    {"question": "How do I contact support?", "correct_answer": "Visit our Help Center or contact customer support."}
]


# Function to compute similarity
def compute_similarity(answer1, answer2):
    embeddings = similarity_model.encode([answer1, answer2])
    return util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()

# Run tests
for qa_test in qa_tests:
    question = qa_test["question"]
    correct_answer = qa_test["correct_answer"]

    print(f"Q: {question}")

    for model_name, qa_pipeline in qa_pipelines.items():
        start_time = time.time()
        model_answer = qa_pipeline({"question": question, "context": shared_context})["answer"]
        elapsed_time = time.time() - start_time

        # Compute similarity
        similarity_score = compute_similarity(model_answer, correct_answer)

        print(f"{model_name} Answer: {model_answer} (Time: {elapsed_time:.4f}s, Similarity: {similarity_score:.4f})")

    print("=" * 80)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Q: How do I reset my password?
BERT Answer: go to 'Account Settings' and select 'Forgot password (Time: 0.3340s, Similarity: 0.9913)
DistilBERT Answer: go to 'Account Settings' and select 'Forgot password (Time: 0.1715s, Similarity: 0.9913)
ALBERT Answer: 'Account Settings' (Time: 0.4269s, Similarity: 0.4796)
Q: How can I change my email address?
BERT Answer: visit 'Account Settings' > 'Edit Profile' (Time: 0.2912s, Similarity: 0.9297)
DistilBERT Answer: visit 'Account Settings' > 'Edit Profile (Time: 0.1297s, Similarity: 0.8974)
ALBERT Answer: 'Account Settings' > 'Edit Profile'. (Time: 0.3690s, Similarity: 0.9395)
Q: Why is my account suspended?
BERT Answer: policy violations (Time: 0.2832s, Similarity: 0.2635)
DistilBERT Answer: policy violations (Time: 0.1795s, Similarity: 0.2635)
ALBERT Answer: policy violations. (Time: 0.3959s, Similarity: 0.2649)
Q: How do I delete my account?
BERT Answer: click 'Delete Account' (Time: 0.2852s, Similarity: 0.8243)
DistilBERT Answer: go to 'Setti